# Overview

This script does more-or-less the same thing as [the singular variant](./gateway_aggregator_analysis_singular.ipynb), however, aggregates the results of different results, and performs trend analysis across different configurations (accessible vCPUs etc.).

In [2]:
import datetime
import json
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr


pincirollis_data_path = "/workspaces/muBench-experiment/gssi_experiment/gateway_aggregator/pincirolli_results.csv"
synth_df = pd.read_csv(pincirollis_data_path, header=0)
synth_df.describe()


results_base_path = "/workspaces/muBench-experiment/gssi_experiment/gateway_aggregator/results/2023_11_13/results/2023_11_12/500m_1rep_25trials/run_1/"

# Request delay

## Dataframe creation

In [3]:
# Generates data from the step data

columns = [
    # Metadata
    "s3_intensity",
    "run_index",
    "replications",
    "trials",
    "cpu_cap",
    # Results latency
    "real_avg_latency_ms",
    "synth_latency_ms",
]
# Results CPU utilization.
services = ['s1', 's2', 's3', 'gateway-aggregator']
columns.extend([f'{service}_real_avg_cpu_utilization' for service in services])
columns.extend([f'{service}_synth_cpu_utilization' for service in services])

df = pd.DataFrame(columns=columns)


# max_steps = 25
# for step in range(max_steps + 1):
#     path = f"{results_base_path}/{step}_steps/mubench_results.csv"
#     if not os.path.exists(path):
#         print(f'Could not load data from path "{path}"')
#         continue
#     exp_df = pd.read_csv(path, header=0)
#     avg_latency = np.average(exp_df["latency_ms"])
#     s3_intensity = 1 - (step / max_steps)
#     df.at[step, "s3_intensity"] = s3_intensity
#     df.at[step, "avg_latency_ms"] = avg_latency

df

,s3_intensity,run_index,replications,trials,cpu_cap,real_avg_latency_ms,synth_latency_ms,s1_real_avg_cpu_utilization,s2_real_avg_cpu_utilization,s3_real_avg_cpu_utilization,gateway-aggregator_real_avg_cpu_utilization,s1_synth_cpu_utilization,s2_synth_cpu_utilization,s3_synth_cpu_utilization,gateway-aggregator_synth_cpu_utilization
